# Plotting pre-processed SST1 RSoXS data
Use pyhyper's new RSoXS plotting shortctus

## Imports

In [ ]:
## Imports
import PyHyperScattering as phs
import pathlib
import sys
import io
import ast
import json
import datetime
import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from tqdm.auto import tqdm
import subprocess
import gc

print(f'Using PyHyperScattering Version: {phs.__version__}')

## Define paths

In [ ]:
notebookPath = pathlib.Path.cwd() 
zarrsPath = notebookPath  # use this if the example data is in the same folder as this notebook
# zarrsPath = pathlib.Path('/nsls2/data/sst/proposals/2024-2/pass-313412/rsoxs_hackathon_2024_data')  # use this if working in the NSLS2 Jupyterhub to access zarrs

## Load cartesian data 

In [ ]:
[f.name for f in zarrsPath.glob('*zarr')]

In [ ]:
DS = xr.open_zarr(zarrsPath.joinpath('cartesian_data_PM6_CF.zarr'))

# Compute any dask coordiantes
for coord_name, coord_data in DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        DS.coords[coord_name] = coord_data.compute()
        
DS

### Detector images

#### Facet plot at several selected energies

#### Single image at selected energy

#### Generate mp4 movie of detector image at all energies

## Load polar data & set plotting config

In [ ]:
[f.name for f in zarrsPath.glob('*zarr')]

In [ ]:
DS = xr.open_zarr(zarrsPath.joinpath('polar_data_PM6_CF.zarr'))

# Compute any dask coordiantes
for coord_name, coord_data in DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        DS.coords[coord_name] = coord_data.compute()

# Select a dataarray (potentially from larger dataset)
pol = 0
data_variable = 'raw_intensity'
DA = DS[data_variable].sel(polarization=pol)

# Set default plot roi & hint settings:
DA.attrs['plot_roi'] = {
    'chi_width': 90,
    'q_range': (0.01, 0.09),
    'energy_range': (280, 295),
    'energy_default': 285,
    'selected_energies': np.array([275, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 335])
}

DA.attrs['plot_hints'] = {
    'I_cmap': 'turbo',
    'xscale': 'log',
    'yscale': 'log'
}

DA

### Integrated scattered intensity (ISI) plots

In [ ]:
# Call ISI function, return ISI figure & axes object
# fig, ax = DA.pt.plot_ISI(save=False)
fig, ax = DA.pt.plot_ISI()

# Plot
plt.show()
plt.close('all')

In [ ]:
# quick plotting of 2D polar images (selected energy) to find q limits

# %matplotlib widget
# plt.close('all')
# sel_DA = DA.sel(energy=285)
# cmin, cmax = sel_DA.quantile([0.01, 0.99]).compute()
# sel_DA.plot(norm=LogNorm(cmin,cmax), cmap=plt.cm.turbo)
# plt.show()

### Intensity heatmaps

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_Imap(save=False)
fig, ax = DA.pt.plot_Imap()

# Plot
plt.show()
plt.close('all')

### Intensity vs Q linecuts

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_IvQ(save=False)
fig, ax = DA.pt.plot_IvQ()

# Plot
plt.show()
plt.close('all')

### Anisotropy heatmaps

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_ARmap(ar_vlim=0.4, save=False)
fig, ax = DA.pt.plot_ARmap(ar_vlim=0.4)

# Plot
plt.show()
plt.close('all')

### Anisotropy vs Q linecuts

# Previous, less organized code below:

## Intensity Plots

### 1. Detector movies

In [ ]:
%matplotlib inline

In [ ]:
plt.close('all')

In [ ]:
already_made_movies = set([f.name[:-17] for f in plotsPath.joinpath('rsoxs_carbon_detector_movies_v1').glob('*pol90*')])

In [ ]:
selected_samples = list(set(DS.sample_name.values).difference(already_made_movies))
selected_samples

In [ ]:
# Parameters
edge = 'carbon'
# DS = rsoxs_datasets[f'cartesian_{edge}']
# DS = rsoxs_datasets[f'cartesian_{edge}_reverse']
intensity_type = 'raw'
pol = 0
energy = 285.2

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
# selected_samples = ['PM6_5CN-CB']

# for sample_name in DS.sample_name.values:
for sample_name in selected_samples:
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    DA = DA.where(DA>0, 10)
    # Make & customize plot
    sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})

    cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.03))
    cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))
    
    # cmin = cmin if cmin > 1e8 else 1e8

    ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
    ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
    ax.figure.set_tight_layout(True)   
    ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
    ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=12)

    plt.show()
    plt.close('all')

In [ ]:
# Select Dataset
# edge = 'carbon'
# DS = rsoxs_datasets[f'cartesian_{edge}_reverse']
# DS = rsoxs_datasets[f'cartesian_{edge}_tilted']

# bcx = DS['raw_intensity'].beamcenter_x
# bcy = DS['raw_intensity'].beamcenter_y

# Select Plotting Parameters
# pol = 90
# pix_size = 500
# pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
# pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
intensity_type = 'raw'

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
# selected_samples = ['PM6_5CN-CB']
for pol in [0, 90]:
    # for sample_name in tqdm(DS.sample_name.values, desc=f'Polarization {pol}°'):
    for sample_name in tqdm(selected_samples, desc=f'Polarization {pol}°'):
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']

        savePath = plotsPath.joinpath('rsoxs_carbon_detector_movies_v1')
        savePath.mkdir(exist_ok=True)
        output_path = savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '16',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '22',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for i, energy in enumerate(tqdm(DA.energy.values, desc=f'Making the {sample_name} movie')):
            # Make & customize plot
            sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})
            
#             cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.05))
#             cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))

#             cmin = cmin if cmin > 1e8 else 1e8
            
            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=12)

            # Save first frame (or specific energy value)
            if energy == 285.2:
                ax.figure.savefig(savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")
            
        gc.collect()  # try to clear up some memory...


## Anisotropy Ratio Plots

### 2. 1D meaned AR linecuts

In [ ]:
# make selection
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'raw'
chi_width = 90
q_slice = slice(0.008, None)
e_slice = slice(282, 292)

selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
                    'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
                    'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 

for sample_name in DS.sample_name.values[:]:
# for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(energies):
        (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log'))

    fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.legend(title='Energy [eV]', loc=(1.03,0.02))

    # savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ar_cuts_v1')
    # savePath.mkdir(exist_ok=True)
    # fig.savefig(savePath.joinpath( 
    #     f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot   
        energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data
        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))        

        # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
        # cax = ax.inset_axes([1.03, 0, 0.03, 1])
        # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        # cbar.set_label(label=f'Time [seconds]', labelpad=14)
        # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

        fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

        for j, energy in enumerate(energies):
            (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log'))

        fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46, y=0.95)

        # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
        ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')

        ax.legend(title='Energy [eV]', loc=(1.03,0.02))

        # savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ar_cuts_v1')
        # savePath.mkdir(exist_ok=True)
        # fig.savefig(savePath.joinpath( 
        #     f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

### Para - perp log scale intensity

In [ ]:
# make selection
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'raw'
chi_width = 90
q_slice = slice(0.01, None)
e_slice = slice(282, 292)
porod_slices = []
# porod_slices = [slice(0.0107, 0.0138), # Calculate 1 degree polynomial (straigh line) through these regions
#                 slice(0.0107, 0.0240),
#                 slice(0.0250, 0.0500),
#                 slice(0.0500, 0.0800)]

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
# selected_samples = ['PM6_5CN-CB', 'PM6_CF']

# plt.close('all')

all_slopes = {}
for sample_name in DS.sample_name.values[:]:
# for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    slopes = {}
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    # ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
    ar_DA = np.abs(para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[270, 283, 284.4, 284.8, 284.9, 285, 285.1, 285.2, 285.3, 285.4, 286.2, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(tqdm(energies, desc='Plotting & fitting slopes per energy')):
        sel_DA = ar_DA.sel(energy=energy)
        sel_DA.plot.line(ax=ax, color=colors[j], label=energy, xscale='log', yscale='log',
                         linestyle='None', marker='.', markersize=3)
        
        porod_slopes = {}
        for porod_slice in porod_slices:
            m, b = np.polyfit(np.log(sel_DA.sel(q=porod_slice).q.data), np.log(sel_DA.sel(q=porod_slice).data.compute()), deg=1)
            # display(porod_slice, m)       
            porod_slopes[f'{porod_slice.start}-{porod_slice.stop}'] = np.round(m, 2)
            
        slopes[energy] = porod_slopes
        
    all_slopes[sample_name] = slopes
    fig.suptitle(f'$\parallel$ - $\perp$ Intensity Absolute Difference: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(1e0, None), ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(None, None), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.axes.grid(visible=True, which='both', axis='x')
    legend = ax.legend(title='Energy [eV]', loc=(1.03,0.02))
    for handle in legend.legend_handles:
        handle.set_markersize(15)
    
    savePath = plotsPath.joinpath('rsoxs_carbon_para-perp_cuts_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')
    
# # Save all_slopes.json in same save folder
# json_data = json.dumps(all_slopes)
# with open(str(savePath.joinpath('all_slopes.json')), 'w') as f:
#     f.write(json_data)